In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd
import numpy as np
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re
import copy
from bs4 import BeautifulSoup
import pickle
import glob
from datetime import timedelta, date
import urllib.request
from multiprocessing import Pool
import time

req_proxy = RequestProxy() #you may get different number of proxy when  you run this at each time
proxies = req_proxy.get_proxy_list() #this will create proxy list

## Step 1: Get html links for all cases

In [2]:
# From 01/01/2020 to 01/01/2021: 288004 cases
# From 01/01/2019 to 01/01/2020: 416211 cases
# From 01/01/2018 to 01/01/2019: 429257 cases
# From 01/01/2017 to 01/01/2018: 439794 cases
# From 01/01/2016 to 01/01/2017: 432717 cases
# From 01/01/2014 to 01/01/2016: 833128 cases
# From 01/01/2012 to 01/01/2014: 790850 cases
# From 01/01/2010 to 01/01/2012: 723020 cases

In [41]:
start_date = '01_01_2010'
end_date = '01_01_2012'

In [42]:
url = 'https://www.leagle.com/leaglesearch?page=1&tod='+end_date.replace('_','/')+'&frd='+start_date.replace('_','/')

In [43]:
url

'https://www.leagle.com/leaglesearch?page=1&tod=01/01/2012&frd=01/01/2010'

In [44]:
chrome_options=Options()
#chrome_options.add_argument('--headless')
#br = webdriver.Chrome(options=chrome_options,executable_path=r'C:\Users\i26184\Desktop\chromedriver.exe')
chrome_options.add_experimental_option("prefs", {
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": False
})
br = webdriver.Chrome(options=chrome_options, executable_path='/home/ubuntu/chromedriver')
br.get(url)

element = WebDriverWait(br, 25).until(EC.presence_of_element_located((By.XPATH, "//div[@class='search-result-1']")))
#page_number = url.replace('https://www.leagle.com/leaglesearch?page=','').replace('&tod=09/10/2020&frd=01/01/1700','')

In [45]:
page_number = url.replace('https://www.leagle.com/leaglesearch?page=','').split('&tod')[0].strip()
file_name = '/home/ubuntu/leagle_all_cases/'+start_date+'_to_'+end_date+'_page_'+page_number+'.html'

with open(file_name, "w") as file:
        file.write(br.page_source)

In [46]:
total_pages = int(723020/20)+1

In [47]:
total_pages

36152

In [ ]:
for i in range(total_pages):
    print(i)
    url_old = br.current_url
    next_btn = br.find_element_by_class_name('next')
    next_btn.click()
    
    time.sleep(1)
    
    url_new = br.current_url

    page_number = url_new.replace('https://www.leagle.com/leaglesearch?page=','').split('&tod')[0].split('&frd')[0].strip()
    file_name = '/home/ubuntu/leagle_all_cases/'+start_date+'_to_'+end_date+'_page_'+page_number+'.html'

    with open(file_name, "w") as file:
            file.write(br.page_source)